In [14]:
import matplotlib.pyplot as plt

In [31]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [33]:
output_notebook()

Loading BokehJS ...

In [7]:
(spark.sql('SHOW DATABASES').show())

+------------+
|databaseName|
+------------+
|     default|
| opengovdata|
|          sm|
|        test|
+------------+



In [8]:
spark.sql('SHOW TABLES FROM opengovdata').show(truncate = False)

+-----------+--------------------------------------+-----------+
|database   |tableName                             |isTemporary|
+-----------+--------------------------------------+-----------+
|opengovdata|civil_society_selected                |false      |
|opengovdata|civil_society_selected_vars_metadata  |false      |
|opengovdata|civil_society_selected_without_padding|false      |
|opengovdata|world_bank_full                       |false      |
|opengovdata|world_bank_selected                   |false      |
|opengovdata|world_bank_selected_vars_metadata     |false      |
|opengovdata|world_bank_selected_without_padding   |false      |
+-----------+--------------------------------------+-----------+



In [9]:
import pandas as pd

world_bank_selected = spark.sql("SELECT * FROM opengovdata.world_bank_selected")

world_bank_selected = world_bank_selected.toPandas()

In [10]:
world_bank_selected.head()

,COUNTRY,YEAR,NY_ADJ_AEDU_GN_ZS,NY_ADJ_ICTR_GN_ZS,NY_ADJ_NNTY_KD_ZG,NY_GDP_TOTL_RT_ZS,AG_LND_ARBL_ZS,IC_BUS_EASE_XQ,IC_EC_QJPI,IC_LGL_DURS,...,BN_CAB_XOKA_GD_ZS,CC_EST,GE_EST,PV_EST,RL_EST,RQ_EST,VA_EST,REGION,SUB_REGION,INTERMEDIATE_REGION
0,Afghanistan,1990.0,1.6,NaN,NaN,1.065727,12.11592,NaN,NaN,NaN,...,-5.915854,NaN,NaN,NaN,NaN,NaN,NaN,Asia,Southern Asia,
1,Afghanistan,1991.0,1.6,NaN,NaN,1.065727,12.11592,NaN,NaN,NaN,...,-5.915854,NaN,NaN,NaN,NaN,NaN,NaN,Asia,Southern Asia,
2,Afghanistan,1992.0,1.6,NaN,NaN,1.065727,12.11592,NaN,NaN,NaN,...,-5.915854,NaN,NaN,NaN,NaN,NaN,NaN,Asia,Southern Asia,
3,Afghanistan,1993.0,1.6,NaN,NaN,1.065727,11.98419,NaN,NaN,NaN,...,-5.915854,NaN,NaN,NaN,NaN,NaN,NaN,Asia,Southern Asia,
4,Afghanistan,1994.0,1.6,NaN,NaN,1.065727,11.79885,NaN,NaN,NaN,...,-5.915854,NaN,NaN,NaN,NaN,NaN,NaN,Asia,Southern Asia,


In [19]:
wb = world_bank_selected

In [11]:
civil_society_selected = spark.sql("SELECT * FROM opengovdata.civil_society_selected")

civil_society_selected = civil_society_selected.toPandas()

In [20]:
cs = civil_society_selected

In [21]:
cs.shape

(6104, 219)

In [22]:
cs.head()

,YEAR,COUNTRY,WAVE_WVS,WAVE_DESC_WVS,RESPONDED_CHURCH_N_WVS,RESPONDED_SPORT_N_WVS,RESPONDED_ARTMUSICEDU_N_WVS,RESPONDED_LABORUNION_N_WVS,RESPONDED_POLITICAL_N_WVS,RESPONDED_ENVIRONMENTAL_N_WVS,...,V2PEEDUEQ_VDEM,V2PEEDUEQ_NR_VDEM,V2PEHEALTH_VDEM,V2PEHEALTH_NR_VDEM,V2PEPRISCH_VDEM,V2PESECSCH_VDEM,V2PETERSCH_VDEM,REGION,SUB_REGION,INTERMEDIATE_REGION
0,1990.0,Afghanistan,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.103151,6.0,-1.514656,6.0,27.76105,14.003685,3.266262,Asia,Southern Asia,
1,1991.0,Afghanistan,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.103151,6.0,-1.514656,6.0,27.82737,14.355790,NaN,Asia,Southern Asia,
2,1992.0,Afghanistan,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.525333,6.0,-2.259946,6.0,27.89368,14.707895,NaN,Asia,Southern Asia,
3,1993.0,Afghanistan,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.987092,6.0,-2.360760,6.0,27.96000,15.060000,NaN,Asia,Southern Asia,
4,1994.0,Afghanistan,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.987092,6.0,-2.360760,6.0,40.67998,15.914286,NaN,Asia,Southern Asia,


In [24]:
cs['COUNTRY'].unique()

array([u'Afghanistan', u'Albania', u'Algeria', u'Andorra', u'Angola',
       u'Anguilla', u'Antigua and Barbuda', u'Argentina', u'Armenia',
       u'Aruba', u'Australia', u'Austria', u'Azerbaijan', u'Bahamas',
       u'Bahrain', u'Bangladesh', u'Barbados', u'Belarus', u'Belgium',
       u'Belize', u'Benin', u'Bermuda', u'Bhutan', u'Bolivia',
       u'Bosnia and Herzegovina', u'Botswana', u'Brazil',
       u'Brunei Darussalam', u'Bulgaria', u'Burkina Faso', u'Myanmar',
       u'Burundi', u'Cambodia', u'Cameroon', u'Canada', u'Cape Verde',
       u'Cayman Islands', u'Central African Republic', u'Chad', u'Chile',
       u'China', u'Colombia', u'Comoros', u'Congo', u'Cook Islands',
       u'Costa Rica', u'Croatia', u'Cuba', u'Cyprus', u'Czech Republic',
       u'Democratic Republic of Congo', u'Denmark', u'Djibouti',
       u'Dominica', u'Dominican Republic', u'Ecuador', u'Egypt',
       u'El Salvador', u'Equatorial Guinea', u'Eritrea', u'Estonia',
       u'Ethiopia', u'Faroe Islands', u'F

In [28]:
cee = ['Hungary', 'Czech Republic', 'Poland']
chp = cs.query('COUNTRY in @cee')

In [35]:
h = chp[chp['COUNTRY'] == 'Hungary']
p = figure()
p.line(h.YEAR, h['v2xcs_ccsi_VDEM'.upper()])
show(p)